In [4]:
import pandas as pd
import anndata as ad
import numpy as np
from scipy import sparse
from sklearn.preprocessing import MultiLabelBinarizer

from typing import (  # Meta  # Generic ABCs  # Generic
    TYPE_CHECKING,
    Any,
    Literal,
)

In [5]:
class pAnnData:
    """
    Class for storing protein and peptide data, with additional relational data between protein and peptide.
    """

    def __init__(self, 
                 prot = None, # np.ndarray | sparse.spmatrix 
                 pep = None, # np.ndarray | sparse.spmatrix
                 rs = None): # np.ndarray | sparse.spmatrix, protein x peptide relational data

        if prot is not None:
            self._prot = ad.AnnData(prot)
        if pep is not None:
            self._pep = ad.AnnData(pep)
        if rs is not None:
            self._set_RS(rs)
            
    @property
    def prot(self):
        return self._prot

    @property
    def pep(self):
        return self._pep
    
    @property
    def rs(self):
        return self._rs

    @prot.setter
    def prot(self, value):
        self._prot = value

    @pep.setter
    def pep(self, value):
        self._pep = value

    @rs.setter
    def rs(self, value):
        self._set_RS(value)

    def _set_RS(self, rs):
        # assert that the dimensions of rs match either protein x peptide or peptide x protein
        assert ((self.prot is None or rs.shape[0] == self.prot.shape[0]) and (self.pep is None or rs.shape[1] == self.pep.shape[0])) or \
            ((self.prot is None or rs.shape[1] == self.prot.shape[0]) and (self.pep is None or rs.shape[0] == self.pep.shape[0])), \
            f"The dimensions of rs ({rs.shape}) must match either protein x peptide ({self.prot.shape[0] if self.prot is not None else None} x {self.pep.shape[0] if self.pep is not None else None}) or peptide x protein ({pep.shape[0] if pep is not None else None} x {prot.shape[0] if prot is not None else None})"

        # check dimensions of rs, make sure protein (row) x peptide (column) format
        if self.prot is not None and rs.shape[0] != self.prot.shape[0]:
            rs = rs.T
        self._rs = sparse.csr_matrix(rs)

In [ ]:
def base_import(prot: np.ndarray, sep: str, prot: str, pep: str, rs: str) -> pAnnData:
    # import protein-peptide data from file
    rs = df[rs].values
    return pAnnData(prot, pep, rs)

def import_proteomeDiscoverer(paths: List[str], data_type: str = None, pdata: pAnnData = None) -> pAnnData:
    # import data from ProteomeDiscoverer
    # check the number of files
    if len(paths) == 1:
        # check whether file name contains 'prot' or 'pep'
        if "prot" in paths[0]:
            df = pd.read_csv(path, sep = sep)
            return base_import(paths[0], sep="\t", prot="prot", pep=None, rs="rs")
        elif "pep" in paths[0]:
            return base_import(paths[0], sep="\t", prot=None, pep="pep", rs="rs")
        else:
            raise ValueError("The file name must contain either 'prot' or 'pep'.")
    elif len(paths) == 2:
        # check whether one file name contains 'prot' and the other contains 'pep'
        if ("prot" in paths[0] and "pep" in paths[1]) or ("prot" in paths[1] and "pep" in paths[0]):
            return base_import(paths, sep="\t", prot="prot", pep="pep", rs="rs")
        else:
            raise ValueError("One file name must contain 'prot' and the other 'pep'.")
    else:
        raise ValueError("Expected 1 or 2 file paths, got {}.".format(len(paths)))

def import_DIA_NN(path: str,  pAnnData: pAnnData, none) -> pAnnData:
    # import data from DIA-NN
    # replace "prot_DIA", "pep_DIA", "rs_DIA" with the actual column names in DIA-NN files
    return base_import(path, sep=",", prot="prot_DIA", pep="pep_DIA", rs="rs_DIA")

# Functions to prepare pAnnData

In [35]:
# PROTEOME DISCOVERER
# -----------------------------
# import pd test files
pep_all = pd.read_csv('pd_pep.txt', sep='\t')
prot_all = pd.read_csv('pd_prot.txt', sep='\t')

# -----------------------------
# PEPTIDE DATA
# pep_X: sparse data matrix
pep_X = sparse.csr_matrix(pep_all.filter(regex='Abundance: F', axis=1).values)
# pep_var: peptide sequence with modifications
pep_var = (pep_all['Annotated Sequence'] + np.where(pep_all['Modifications'].isna(), '', ' MOD:' + pep_all['Modifications'])).values
# pep_obs: file names
pep_obs = pep_all.filter(regex='Abundance: F', axis=1).columns.str.extract('Abundance: (F\d+):')[0].values

# -----------------------------
# PROTEIN DATA
# prot_X: sparse data matrix
prot_X = sparse.csr_matrix(prot_all.filter(regex='Abundance: F', axis=1).values)
# prot_var: protein names
prot_var = prot_all['Accession'].values
# prot_obs: file names
prot_obs = prot_all.filter(regex='Abundance: F', axis=1).columns.str.extract('Abundance: (F\d+):')[0].values

# -----------------------------
# RS DATA
pep_prot_list = pep_all['Master Protein Accessions'].str.split('; ')
mlb = MultiLabelBinarizer()
rs = mlb.fit_transform(pep_prot_list)
index_dict = {protein: index for index, protein in enumerate(mlb.classes_)}
reorder_indices = [index_dict[protein] for protein in prot_var]
rs = rs[:, reorder_indices]

# ASSERTIONS
# -----------------------------
# check that all files overlap, and that the order is the same
pep_obs_set = set(pep_obs)
prot_obs_set = set(prot_obs)
assert pep_obs_set == prot_obs_set, "The files in peptide and protein data must be the same"
# -----------------------------
# check if mlb.classes_ has overlap with prot_var
mlb_classes_set = set(mlb.classes_)
prot_var_set = set(prot_var)

if mlb_classes_set != prot_var_set:
    print("WARNING: Master proteins in the peptide matrix do not match proteins in the protein data")
    # print numerical overlap, and unique elements in each set
    print(f"Overlap: {len(mlb_classes_set & prot_var_set)}")
    print(f"Unique to peptide data: {mlb_classes_set - prot_var_set}")
    print(f"Unique to protein data: {prot_var_set - mlb_classes_set}")
# -----------------------------
# # export protein_peptide_matrix as csv
# df = pd.DataFrame(protein_peptide_matrix, columns=prot_var, index=pep_var)
# df.to_csv('protein_peptide_matrix.csv')

test = pAnnData(prot_X, pep_X, rs)
test.rs

<1571x6352 sparse matrix of type '<class 'numpy.int32'>'
	with 6959 stored elements in Compressed Sparse Row format>

In [36]:
pep_prot_list

0                                                [P36578]
1                                                [P05388]
2                                                [P04264]
3                                                [P35908]
4                                                [Q15637]
                              ...                        
6347                                             [P14314]
6348                                             [Q06830]
6349                                             [Q2TAA2]
6350    [P02533, Q15323, P08727, P13645, P05783, Q0469...
6351                                             [P49411]
Name: Master Protein Accessions, Length: 6352, dtype: object

In [27]:
# DIA-NN

# "Run" = file names
# "Precursor.Id" = peptide unique identifier 
# "Precursor.Translated" = peptide abundance
# "Protein.Group" = protein unique identifier
# "PG.MaxLFQ" = protein abundance
# "Genes" = gene name 

# ???? things to consider:
# figure out what the difference is btw PG.MaxLFQ and Genes.MaxLFQ.Unique - ask some DIANN ppl maybe

# -----------------------------
# import DIA-NN test file
report_all = pd.read_csv('report.tsv', sep='\t')

# -----------------------------
# PEPTIDE DATA
# pep_X: sparse data matrix
pep_X_pivot = report_all.pivot_table(index='Precursor.Id', columns='Run', values='Precursor.Translated', aggfunc='first')
pep_X_pivot.fillna(0, inplace=True)
pep_X = sparse.csr_matrix(pep_X_pivot.values)
# pep_var: peptide sequence
pep_var = pep_X_pivot.index.values
# pep_obs: file names
pep_obs = pep_X_pivot.columns.values

# -----------------------------
# PROTEIN DATA
# prot_X: sparse data matrix
prot_X_pivot = report_all.pivot_table(index='Protein.Group', columns='Run', values='PG.MaxLFQ', aggfunc='first')
prot_X_pivot.fillna(0, inplace=True)
prot_X = sparse.csr_matrix(prot_X_pivot.values)
# prot_var: protein names
prot_var = prot_X_pivot.index.values
# prot_obs: file names
prot_obs = prot_X_pivot.columns.values


In [30]:
# -----------------------------
# RS DATA
# rs: protein x peptide relational data
pep_prot_list = report_all['Protein.Ids'].str.split(';')
mlb = MultiLabelBinarizer()
rs = mlb.fit_transform(pep_prot_list)
index_dict = {protein: index for index, protein in enumerate(mlb.classes_)}

In [33]:
rs = rs[:, reorder_indices]

In [34]:
pep_prot_list

0                                          [P37108, H0YLW0]
1                                          [P37108, H0YLW0]
2                                          [P37108, H0YLW0]
3                                          [P37108, H0YLW0]
4                                          [P37108, H0YLW0]
                                ...                        
189850    [Q8N183, A0A7I2V3X5, A0A7I2V4Z7, A0A7I2YQX2, D...
189851                                             [O00257]
189852                                             [O00257]
189853             [P50851, A0A494C0R9, A0A494C1L5, E9PEM5]
189854             [P50851, A0A494C0R9, A0A494C1L5, E9PEM5]
Name: Protein.Ids, Length: 189855, dtype: object

In [20]:
# check if PG.MaxLFQ column is same as Genes.MaxLFQ.Unique
# assert (report_all['PG.MaxLFQ'] == report_all['Genes.MaxLFQ.Unique']).all(), "The protein abundance columns do not match"

# show rows where PG.MaxLFQ != Genes.MaxLFQ.Unique
report_all[report_all['PG.MaxLFQ'] != report_all['Genes.MaxLFQ.Unique']]

,File.Name,Run,Protein.Group,Protein.Ids,Protein.Names,Genes,PG.Quantity,PG.Normalised,PG.MaxLFQ,Genes.Quantity,...,Decoy.Evidence,Decoy.CScore,Fragment.Quant.Raw,Fragment.Quant.Corrected,Fragment.Correlations,MS2.Scan,IM,iIM,Predicted.IM,Predicted.iIM
51,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela200ng_LysC_20240321_QE_DDA_Mechtler...,Q6ZUT4,Q6ZUT4,YL014_HUMAN,NaN,516013.0,172732.0,172732.0,NaN,...,0.00000,-1.000000e+07,142886;167621;90188.2;205507;205075;0;0;3009.3...,142886;167621;90188.2;205507;205075;0;0;3009.3...,0.866537;0.798672;0.784366;0.822582;0.47907;0;...,11832,0,0,0,0
90,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela20ng_LysC_20240321_QE_DIA_Mechtler_...,Q9Y490,Q9Y490,TLN1_HUMAN,TLN1,1190840.0,1009230.0,1198350.0,1190840.0,...,0.00000,-1.000000e+07,7171.93;6207.61;5829.97;3511.98;1306.93;1340.43;,7171.93;6207.61;5829.97;3511.98;1306.93;1340.43;,0.816497;0.816497;0.816497;0.816497;0.816497;0...,8642,0,0,0,0
91,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela200ng_LysC_20240321_QE_DIA_Mechtler...,Q9Y490,Q9Y490,TLN1_HUMAN,TLN1,3726720.0,1779030.0,1152090.0,3726720.0,...,0.00000,-1.000000e+07,12516.1;12034.9;8982.22;12280;3453.99;2164.27;,12516.1;12034.9;8982.22;12280;3453.99;2164.27;,0.816497;0.904415;0.816497;0.912802;0.816497;0...,9191,0,0,0,0
92,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela20ng_LysC_20240321_QE_DIA_Mechtler_...,Q9Y490,Q9Y490,TLN1_HUMAN,TLN1,1587110.0,1106420.0,1698530.0,1587110.0,...,0.00000,-1.000000e+07,6620.8;5662.74;5482.44;6842.59;2535.6;1017.9;,6620.8;5662.74;5482.44;6842.59;2535.6;1017.9;,0.816497;0.816497;0.816497;0.912663;0.693105;0...,8520,0,0,0,0
93,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela2ng_LysC_20240321_QE_DIA_Mechtler_A...,Q9Y490,Q9Y490,TLN1_HUMAN,TLN1,862847.0,1770910.0,1569280.0,862847.0,...,0.00000,-1.000000e+07,9438.69;7991.5;8815.02;4665.09;1873.37;2416.96;,9438.69;7991.5;8815.02;4665.09;1873.37;2416.96;,0.816497;0.816497;0.816497;0.816497;0.816497;0...,8337,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189838,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela20ng_LysC_20240321_QE_DIA_Mechtler_...,P68104;Q5VTE0,P68104;Q5VTE0;A0A087WV01;A0A087WVQ9;A0A7I2V3H3...,EF1A1_HUMAN;EF1A3_HUMAN,EEF1A1;EEF1A1P5,175790000.0,136659000.0,54324900.0,175790000.0,...,0.00000,-1.000000e+07,1.91415e+06;2.3898e+06;1.95667e+06;1.83351e+06...,1.91415e+06;2.3898e+06;1.95667e+06;1.83351e+06...,0.816497;0.816497;0.816497;0.816497;0.816497;0;,10984,0,0,0,0
189839,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela2ng_LysC_20240321_QE_DIA_Mechtler_A...,P68104;Q5VTE0,P68104;Q5VTE0;A0A087WV01;A0A087WVQ9;A0A7I2V3H3...,EF1A1_HUMAN;EF1A3_HUMAN,EEF1A1;EEF1A1P5,14487900.0,20458800.0,29510200.0,14487900.0,...,0.00000,-1.000000e+07,1.4728e+06;1.39577e+06;522764;793315;410962;21...,1.4728e+06;1.39577e+06;522764;793315;410962;21...,0.816497;0.816497;0.816497;0.984607;0.816497;0...,10557,0,0,0,0
189840,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela200pg_LysC_20240321_QE_DIA_Mechtler...,P68104;Q5VTE0,P68104;Q5VTE0;A0A087WV01;A0A087WVQ9;A0A7I2V3H3...,EF1A1_HUMAN;EF1A3_HUMAN,EEF1A1;EEF1A1P5,13635900.0,46175100.0,53102600.0,13635900.0,...,0.00000,-1.000000e+07,463033;0;0;1.88557e+06;0;0;,463033;0;0;1.88557e+06;0;0;,0.816497;0;0;0.879735;0;0;,10130,0,0,0,0
189853,D:\Caltech\caltech-edu_mpangwan\dia-nn\raw\Mar...,Marion_Hela20ng_LysC_20240321_QE_DIA_Mechtler_...,P50851,P50851;A0A494C0R9;A0A494C1L5;E9PEM5,LRBA_HUMAN,LRBA,197255.0,166343.0,214200.0,197255.0,...,0.00000,-1.000000e+07,47066.6;0;23296;35825.7;22960.5;2909.91;0;0;0;...,47066.6;0;23296;35825.7;22960.5;2909.91;0;0;0;...,0.863027;0;0.816497;0.858623;0.816497;0;0;0;0;...,10576,0,0,0,0
